In [ ]:
import requests
import pandas as pd
from datetime import datetime
import time
import os  # For file paths

API_KEY = "nopee"  # Your API key

# Full list of 64 districts
DISTRICTS = [
    "Bagerhat,Bangladesh", "Bandarban,Bangladesh", "Barguna,Bangladesh", "Barisal,Bangladesh", "Bhola,Bangladesh",
    "Bogra,Bangladesh", "Brahmanbaria,Bangladesh", "Chandpur,Bangladesh", "Chittagong,Bangladesh", "Chuadanga,Bangladesh",
    "Comilla,Bangladesh", "Cox's Bazar,Bangladesh", "Dhaka,Bangladesh", "Dinajpur,Bangladesh", "Faridpur,Bangladesh",
    "Feni,Bangladesh", "Gaibandha,Bangladesh", "Gazipur,Bangladesh", "Gopalganj,Bangladesh", "Habiganj,Bangladesh",
    "Jamalpur,Bangladesh", "Jessore,Bangladesh", "Jhalokati,Bangladesh", "Jhenaidah,Bangladesh", "Joypurhat,Bangladesh",
    "Khagrachhari,Bangladesh", "Khulna,Bangladesh", "Kishoreganj,Bangladesh", "Kurigram,Bangladesh", "Kushtia,Bangladesh",
    "Lakshmipur,Bangladesh", "Lalmonirhat,Bangladesh", "Madaripur,Bangladesh", "Magura,Bangladesh", "Manikganj,Bangladesh",
    "Meherpur,Bangladesh", "Moulvibazar,Bangladesh", "Munshiganj,Bangladesh", "Mymensingh,Bangladesh", "Naogaon,Bangladesh",
    "Narail,Bangladesh", "Narayanganj,Bangladesh", "Narsingdi,Bangladesh", "Natore,Bangladesh", "Nawabganj,Bangladesh",
    "Netrakona,Bangladesh", "Nilphamari,Bangladesh", "Noakhali,Bangladesh", "Pabna,Bangladesh", "Panchagarh,Bangladesh",
    "Patuakhali,Bangladesh", "Pirojpur,Bangladesh", "Rajbari,Bangladesh", "Rajshahi,Bangladesh", "Rangamati,Bangladesh",
    "Rangpur,Bangladesh", "Satkhira,Bangladesh", "Shariatpur,Bangladesh", "Sherpur,Bangladesh", "Sirajganj,Bangladesh",
    "Sunamganj,Bangladesh", "Sylhet,Bangladesh", "Tangail,Bangladesh", "Thakurgaon,Bangladesh"
]

# Date range (adjusted to 2025 only, as per your URL)
START_YEAR = 2025
END_YEAR = 2025

# Function to fetch data (updated for your 2025 range)
def fetch_weather_year(location, year):
    start_date = f"{year}-01-01"
    end_date = f"{year}-11-20"
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{location}/{start_date}/{end_date}?unitGroup=metric&include=days&key={API_KEY}&contentType=json"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        days = data.get("days", [])
        if not days:
            print(f"No data for {location} {year}")
        else:
            print(f"Fetched {len(days)} days for {location} {year}")
        return days
    except Exception as e:
        print(f"Error for {location} {year}: {str(e)}")
        return []

# Choose districts to fetch (all for full run; or ["Bagerhat,Bangladesh"] for just one)
districts_to_fetch = DISTRICTS  # Or DISTRICTS[0:1] for testing with Bagherhat

for location in districts_to_fetch:
    print(f"\nStarting district: {location}")
    all_data = []  # Reset for each district
    
    for year in range(START_YEAR, END_YEAR + 1):
        year_data = fetch_weather_year(location, year)
        all_data.extend(year_data)
        time.sleep(2)  # Optional delay to avoid rate limits
    
    # Process the data
    extracted_data = []
    for day in all_data:
        date_str = day.get("datetime")
        if date_str:
            dt = datetime.strptime(date_str, "%Y-%m-%d")
            if dt > datetime(2025, 11, 20):  # Filter beyond end date if needed
                continue
            day_val = dt.day
            month_val = dt.month
            year_val = dt.year
        else:
            day_val = month_val = year_val = None
        
        row = {
            "district": location.split(",")[0],
            "day": day_val,
            "month": month_val,
            "year": year_val,
            "temperature(degree C)": day.get("temp"),
            "feels_like(degree C)": day.get("feelslike"),
            "max_temperature(degree C)": day.get("tempmax"),
            "minimum_temperature(degree C)": day.get("tempmin"),
            "humidity": day.get("humidity"),
            "precipitation": day.get("precip"),
            "windspeed": day.get("windspeed"),
            "atmospheric_pressure": day.get("pressure"),
            "UV": day.get("uvindex"),
            "solar_radiation": day.get("solarradiation"),
            "dew_point": day.get("dew")
        }
        extracted_data.append(row)
    
    # Save per district
    df = pd.DataFrame(extracted_data)
    filename = f"{location.split(',')[0]}_historical_weather_2025.csv"
    if os.path.exists(filename):
        print(f"Skipping {location}, file exists")
        continue
    df.to_csv(filename, index=False)
    print(f"Saved {len(df)} records for {location} to {filename}")


Starting district: Bagerhat,Bangladesh
Error for Bagerhat,Bangladesh 2025: 401 Client Error:  for url: https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Bagerhat,Bangladesh/2025-01-01/2025-11-20?unitGroup=metric&include=days&key=nopee&contentType=json
Saved 0 records for Bagerhat,Bangladesh to Bagerhat_historical_weather_2025.csv

Starting district: Bandarban,Bangladesh
Error for Bandarban,Bangladesh 2025: 401 Client Error:  for url: https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Bandarban,Bangladesh/2025-01-01/2025-11-20?unitGroup=metric&include=days&key=nopee&contentType=json


KeyboardInterrupt: 